Deliverable 1 Instructions
1. Generate a set of 2,000 random latitudes and longitudes
2. Retrieve the nearest city using the `citipy` module
3. Perform an API call with the OpenWeatherMap with below information:
    - Latitude and longitude
    - Maximum temperature
    - Percent humidity
    - Percent cloudiness
    - Wind speed
    - Weather description (for example, clouds, fog, light rain, clear sky)
4. Add the Data to a new DataFrame
5. Export DataFrame as a CSV file

In [11]:
# Import the dependencies.
import pandas as pd
import numpy as np

## 1. Generate a set of 2,000 random latitudes and longitudes

In [12]:
# Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [13]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
coordinates

[(-54.965306620802664, 160.51412028705266),
 (14.715533983892186, -157.95697422394653),
 (-79.73413448543425, -128.99176848310512),
 (-1.8980814457580237, -174.8965165524975),
 (76.46141931265242, -137.97465003994913),
 (-2.981283878803154, 1.8215617074299075),
 (68.91380449183768, 63.29134694075677),
 (74.49957248045354, 166.2418026272872),
 (81.29177128065476, -138.20289235319825),
 (-67.94325356720304, -37.141474096064485),
 (-55.042668378678044, -10.569617507879514),
 (5.099827032700588, -94.97219057005586),
 (71.0663064137467, -75.1383741221955),
 (-35.10134539036311, 119.99561519393183),
 (-85.50193414722244, 7.071063520556464),
 (14.115817825432273, -140.359351737827),
 (-27.676619393676148, 60.73914262226984),
 (-16.209113659596113, 91.87178078535891),
 (-9.890547394748054, -128.57943913629458),
 (-13.47527966324067, -32.78354113473375),
 (-56.97863708584721, -97.5991746812953),
 (-37.653035818426716, 169.0158367242562),
 (-16.77029701268455, 82.07082890539743),
 (-84.364095222

## 2. Retrieve the nearest city using the `citipy` module

In [14]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [15]:
# Create a list for holding the cities.
cities = []
# Identify th enearest city for each atitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

754

## 3. Perform an API call with the OpenWeatherMap

In [16]:
# Import the datetime module from the datetime library.
from datetime import datetime
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key

In [17]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=a5e27cecc4fcf20d888a30c63791d839


In [18]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_descp = city_weather["weather"]["description"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Weather Description": city_descp,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | tuatapere
City not found. Skipping...
Processing Record 2 of Set 1 | hilo
City not found. Skipping...
Processing Record 3 of Set 1 | rikitea
City not found. Skipping...
Processing Record 4 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 5 of Set 1 | tuktoyaktuk
City not found. Skipping...
Processing Record 6 of Set 1 | port-gentil
City not found. Skipping...
Processing Record 7 of Set 1 | komsomolskiy
City not found. Skipping...
Processing Record 8 of Set 1 | pevek
City not found. Skipping...
Processing Record 9 of Set 1 | ushuaia
City not found. Skipping...
Processing Record 10 of Set 1 | cape town
City not found. Skipping...
Processing Record 11 of Set 1 | puerto ayora
City not found. Skipping...
Processing Record 12 of Set 1 | clyde river
City not found. Skipping...
Processing Record 13 of Set 1 | albany
City not found. Skipping...
Processing Record 14 of Set 1 | hermanus
City

City not found. Skipping...
Processing Record 19 of Set 3 | porto velho
City not found. Skipping...
Processing Record 20 of Set 3 | ilulissat
City not found. Skipping...
Processing Record 21 of Set 3 | castro
City not found. Skipping...
Processing Record 22 of Set 3 | asau
City not found. Skipping...
Processing Record 23 of Set 3 | boende
City not found. Skipping...
Processing Record 24 of Set 3 | torbay
City not found. Skipping...
Processing Record 25 of Set 3 | fez
City not found. Skipping...
Processing Record 26 of Set 3 | florida
City not found. Skipping...
Processing Record 27 of Set 3 | high level
City not found. Skipping...
Processing Record 28 of Set 3 | berlevag
City not found. Skipping...
Processing Record 29 of Set 3 | samur
City not found. Skipping...
Processing Record 30 of Set 3 | mys shmidta
City not found. Skipping...
Processing Record 31 of Set 3 | isilkul
City not found. Skipping...
Processing Record 32 of Set 3 | chabahar
City not found. Skipping...
Processing Record

City not found. Skipping...
Processing Record 37 of Set 5 | norman wells
City not found. Skipping...
Processing Record 38 of Set 5 | nanortalik
City not found. Skipping...
Processing Record 39 of Set 5 | mwanza
City not found. Skipping...
Processing Record 40 of Set 5 | zdvinsk
City not found. Skipping...
Processing Record 41 of Set 5 | nouadhibou
City not found. Skipping...
Processing Record 42 of Set 5 | itaituba
City not found. Skipping...
Processing Record 43 of Set 5 | hami
City not found. Skipping...
Processing Record 44 of Set 5 | kodiak
City not found. Skipping...
Processing Record 45 of Set 5 | balkhash
City not found. Skipping...
Processing Record 46 of Set 5 | chuy
City not found. Skipping...
Processing Record 47 of Set 5 | mortka
City not found. Skipping...
Processing Record 48 of Set 5 | bethel
City not found. Skipping...
Processing Record 49 of Set 5 | esperance
City not found. Skipping...
Processing Record 50 of Set 5 | malinovskiy
City not found. Skipping...
Processing 

City not found. Skipping...
Processing Record 6 of Set 8 | pangai
City not found. Skipping...
Processing Record 7 of Set 8 | college place
City not found. Skipping...
Processing Record 8 of Set 8 | bogorodskoye
City not found. Skipping...
Processing Record 9 of Set 8 | ambilobe
City not found. Skipping...
Processing Record 10 of Set 8 | nicoya
City not found. Skipping...
Processing Record 11 of Set 8 | pangkalanbuun
City not found. Skipping...
Processing Record 12 of Set 8 | almaznyy
City not found. Skipping...
Processing Record 13 of Set 8 | saint-augustin
City not found. Skipping...
Processing Record 14 of Set 8 | ponta do sol
City not found. Skipping...
Processing Record 15 of Set 8 | tilichiki
City not found. Skipping...
Processing Record 16 of Set 8 | bambous virieux
City not found. Skipping...
Processing Record 17 of Set 8 | dunmanway
City not found. Skipping...
Processing Record 18 of Set 8 | port moresby
City not found. Skipping...
Processing Record 19 of Set 8 | sabang
City no

City not found. Skipping...
Processing Record 24 of Set 10 | sao borja
City not found. Skipping...
Processing Record 25 of Set 10 | sharanga
City not found. Skipping...
Processing Record 26 of Set 10 | fountain
City not found. Skipping...
Processing Record 27 of Set 10 | korem
City not found. Skipping...
Processing Record 28 of Set 10 | presidencia roque saenz pena
City not found. Skipping...
Processing Record 29 of Set 10 | vestmanna
City not found. Skipping...
Processing Record 30 of Set 10 | ca mau
City not found. Skipping...
Processing Record 31 of Set 10 | kudahuvadhoo
City not found. Skipping...
Processing Record 32 of Set 10 | launceston
City not found. Skipping...
Processing Record 33 of Set 10 | portland
City not found. Skipping...
Processing Record 34 of Set 10 | mahibadhoo
City not found. Skipping...
Processing Record 35 of Set 10 | turan
City not found. Skipping...
Processing Record 36 of Set 10 | san lorenzo
City not found. Skipping...
Processing Record 37 of Set 10 | dzao

City not found. Skipping...
Processing Record 44 of Set 12 | burns lake
City not found. Skipping...
Processing Record 45 of Set 12 | taylors
City not found. Skipping...
Processing Record 46 of Set 12 | ginda
City not found. Skipping...
Processing Record 47 of Set 12 | newton
City not found. Skipping...
Processing Record 48 of Set 12 | witu
City not found. Skipping...
Processing Record 49 of Set 12 | nsanje
City not found. Skipping...
Processing Record 50 of Set 12 | lianzhou
City not found. Skipping...
Processing Record 1 of Set 13 | nurota
City not found. Skipping...
Processing Record 2 of Set 13 | college
City not found. Skipping...
Processing Record 3 of Set 13 | ormara
City not found. Skipping...
Processing Record 4 of Set 13 | beloha
City not found. Skipping...
Processing Record 5 of Set 13 | atambua
City not found. Skipping...
Processing Record 6 of Set 13 | santa barbara
City not found. Skipping...
Processing Record 7 of Set 13 | staunton
City not found. Skipping...
Processing R

City not found. Skipping...
Processing Record 12 of Set 15 | ascension
City not found. Skipping...
Processing Record 13 of Set 15 | sahrak
City not found. Skipping...
Processing Record 14 of Set 15 | vallenar
City not found. Skipping...
Processing Record 15 of Set 15 | la pintada
City not found. Skipping...
Processing Record 16 of Set 15 | kenner
City not found. Skipping...
Processing Record 17 of Set 15 | tambacounda
City not found. Skipping...
Processing Record 18 of Set 15 | marsh harbour
City not found. Skipping...
Processing Record 19 of Set 15 | tawnat
City not found. Skipping...
Processing Record 20 of Set 15 | kandrian
City not found. Skipping...
Processing Record 21 of Set 15 | vikulovo
City not found. Skipping...
Processing Record 22 of Set 15 | hobyo
City not found. Skipping...
Processing Record 23 of Set 15 | mikhaylov
City not found. Skipping...
Processing Record 24 of Set 15 | teavaro
City not found. Skipping...
Processing Record 25 of Set 15 | buraydah
City not found. Sk